## **Real-Time Fraud Detection System : ML Model Development**

---

### Description
This Jupyter Notebook serves as a precursor to the Fraud_Detection_System Notebook. The Random Forest Classifier which will be used to identify potential fraud transactions is trained, tested & exported through this Notebook. For the Model development, the data pipeline is designed is designed as follows - 
- Training Data is imported from HDFS
- Spark MLlib is used to train & test the ML Model
- The Pipeline object with the encoders & model is exported back to HDFS

<center></center>

### Notebook Environmnet Setup

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, udf
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
from pyspark.ml.feature import VectorAssembler, StringIndexer, StandardScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
import json
import pandas as pd
from imblearn.over_sampling import SMOTE
import numpy

In [3]:
#creating the Spark Session

spark = SparkSession.builder \
    .appName("Real-Time Fraud Detection") \
    .getOrCreate()
print(f"This cluster relies on Spark '{spark.version}'")

25/03/08 10:46:29 WARN Utils: Your hostname, osbdet resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s1)
25/03/08 10:46:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/08 10:46:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


This cluster relies on Spark '3.5.0'


### Data Preprocessing & Feature Engineering

In [4]:
data = spark.read\
            .option("header", "true")\
            .option("inferSchema", "true")\
            .csv("hdfs://localhost:9000/datalake/raw/fraud-detection/Fraud_transactions_dataset.csv")

In [4]:
data = data.drop("isFlaggedFraud", "step")

In [5]:
data.printSchema()

root
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)



In [6]:
data = data.withColumn("balance_change_orig", col("oldbalanceOrg") - col("newbalanceOrig")) \
                   .withColumn("balance_change_dest", col("newbalanceDest") - col("oldbalanceDest")) \
                   .withColumn("dest_balance_zero", when(col("oldbalanceDest") == 0, 1).otherwise(0)) \
                   .withColumn("is_large_transaction", when(col("amount") > 10000, 1).otherwise(0))\
                   .limit(100000)

### Feature Scaling

In [7]:
#converting transaction type to numerical values
indexer = StringIndexer(inputCol="type", outputCol="type_indexed", handleInvalid="keep")
encoder = indexer.fit(data.drop("isFraud"))
data = encoder.transform(data)

In [8]:
data = data.drop("type")

In [9]:
data.printSchema()

root
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- balance_change_orig: double (nullable = true)
 |-- balance_change_dest: double (nullable = true)
 |-- dest_balance_zero: integer (nullable = false)
 |-- is_large_transaction: integer (nullable = false)
 |-- type_indexed: double (nullable = false)



### Rebalancing Dataset

In [10]:
data_pandas = data.toPandas()

X = data_pandas.drop(columns = ["isFraud", "nameOrig", "nameDest"])
y = data_pandas["isFraud"]


#applying SMOTE to balance fraud cases
smote = SMOTE(sampling_strategy=0.5, random_state=42) 
X_resampled, y_resampled = smote.fit_resample(X, y)

#joining the features & label columns
balanced_data_pandas = pd.DataFrame(X_resampled, columns=X.columns)
balanced_data_pandas["isFraud"] = y_resampled

#converting Pandas to Spark DataFrame
balanced_data = spark.createDataFrame(balanced_data_pandas)

In [11]:
balanced_data.printSchema()

root
 |-- amount: double (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- balance_change_orig: double (nullable = true)
 |-- balance_change_dest: double (nullable = true)
 |-- dest_balance_zero: long (nullable = true)
 |-- is_large_transaction: long (nullable = true)
 |-- type_indexed: double (nullable = true)
 |-- isFraud: long (nullable = true)



### Model Training & Testing

In [12]:
#defining the Vector Assembler
assembler = VectorAssembler(\
                inputCols= ["type_indexed", "amount", "balance_change_orig", "balance_change_dest", "dest_balance_zero", "is_large_transaction"], 
                outputCol="features")

#defining & Training Random Forest Model
rf = RandomForestClassifier(featuresCol="features", labelCol="isFraud", numTrees=50)

In [13]:
train_data, test_data = balanced_data.randomSplit([0.9,0.1], seed=39)

In [14]:
pipeline = Pipeline(stages=[assembler, rf])

#training the Pipeline Model
pipeline_model = pipeline.fit(train_data)

25/03/06 09:12:46 WARN TaskSetManager: Stage 8 contains a task of very large size (5923 KiB). The maximum recommended task size is 1000 KiB.
25/03/06 09:12:48 WARN TaskSetManager: Stage 11 contains a task of very large size (5923 KiB). The maximum recommended task size is 1000 KiB.
25/03/06 09:12:48 WARN TaskSetManager: Stage 12 contains a task of very large size (5923 KiB). The maximum recommended task size is 1000 KiB.
25/03/06 09:12:49 WARN TaskSetManager: Stage 13 contains a task of very large size (5923 KiB). The maximum recommended task size is 1000 KiB.
25/03/06 09:12:49 WARN TaskSetManager: Stage 15 contains a task of very large size (5923 KiB). The maximum recommended task size is 1000 KiB.
25/03/06 09:12:51 WARN TaskSetManager: Stage 17 contains a task of very large size (5923 KiB). The maximum recommended task size is 1000 KiB.
25/03/06 09:12:52 WARN TaskSetManager: Stage 19 contains a task of very large size (5923 KiB). The maximum recommended task size is 1000 KiB.
25/03/0

In [15]:
tdf = test_data.where(test_data["isFraud"] == 1)

In [30]:
predictions = pipeline_model.transform(test_data.limit(500))

In [31]:
predictions.toPandas().tail(10)

25/03/06 09:14:54 WARN TaskSetManager: Stage 104 contains a task of very large size (5923 KiB). The maximum recommended task size is 1000 KiB.


,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,balance_change_orig,balance_change_dest,dest_balance_zero,is_large_transaction,type_indexed,isFraud,features,rawPrediction,probability,prediction
490,2126.05,83499.81,81373.76,0.0,0.0,2126.05,0.0,1,0,0.0,0,"[0.0, 2126.05, 2126.050000000003, 0.0, 1.0, 0.0]","[48.86625643802712, 1.133743561972871]","[0.9773251287605426, 0.022674871239457425]",0.0
491,2130.26,3829.00,1698.74,0.0,0.0,2130.26,0.0,1,0,0.0,0,"[0.0, 2130.26, 2130.26, 0.0, 1.0, 0.0]","[48.86625643802712, 1.133743561972871]","[0.9773251287605426, 0.022674871239457425]",0.0
492,2132.79,0.00,0.00,0.0,0.0,0.00,0.0,1,0,0.0,0,"(0.0, 2132.79, 0.0, 0.0, 1.0, 0.0)","[49.398011774803194, 0.601988225196799]","[0.987960235496064, 0.012039764503935981]",0.0
493,2134.72,0.00,0.00,0.0,0.0,0.00,0.0,1,0,0.0,0,"(0.0, 2134.72, 0.0, 0.0, 1.0, 0.0)","[49.398011774803194, 0.601988225196799]","[0.987960235496064, 0.012039764503935981]",0.0
494,2139.02,20701.00,18561.98,0.0,0.0,2139.02,0.0,1,0,0.0,0,"[0.0, 2139.02, 2139.0200000000004, 0.0, 1.0, 0.0]","[48.86625643802712, 1.133743561972871]","[0.9773251287605426, 0.022674871239457425]",0.0
495,2140.78,643223.00,641082.22,0.0,0.0,2140.78,0.0,1,0,0.0,0,"[0.0, 2140.78, 2140.780000000028, 0.0, 1.0, 0.0]","[48.86625643802712, 1.133743561972871]","[0.9773251287605426, 0.022674871239457425]",0.0
496,2142.13,91105.15,88963.02,0.0,0.0,2142.13,0.0,1,0,0.0,0,"[0.0, 2142.13, 2142.12999999999, 0.0, 1.0, 0.0]","[48.86625643802712, 1.133743561972871]","[0.9773251287605426, 0.022674871239457425]",0.0
497,2144.11,4355.00,2210.89,0.0,0.0,2144.11,0.0,1,0,0.0,0,"[0.0, 2144.11, 2144.11, 0.0, 1.0, 0.0]","[48.86625643802712, 1.133743561972871]","[0.9773251287605426, 0.022674871239457425]",0.0
498,2146.11,0.00,0.00,0.0,0.0,0.00,0.0,1,0,0.0,0,"(0.0, 2146.11, 0.0, 0.0, 1.0, 0.0)","[49.398011774803194, 0.601988225196799]","[0.987960235496064, 0.012039764503935981]",0.0
499,2149.38,763143.16,760993.78,0.0,0.0,2149.38,0.0,1,0,0.0,0,"[0.0, 2149.38, 2149.3800000000047, 0.0, 1.0, 0.0]","[48.86625643802712, 1.133743561972871]","[0.9773251287605426, 0.022674871239457425]",0.0


In [32]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

#Accuracy Evaluator
accuracy_eval = MulticlassClassificationEvaluator(labelCol="isFraud", predictionCol="prediction", metricName="accuracy")

#Precision Evaluator
precision_eval = MulticlassClassificationEvaluator(labelCol="isFraud", predictionCol="prediction", metricName="weightedPrecision")

#Recall Evaluator
recall_eval = MulticlassClassificationEvaluator(labelCol="isFraud", predictionCol="prediction", metricName="weightedRecall")

#F1-Score Evaluator
f1_eval = MulticlassClassificationEvaluator(labelCol="isFraud", predictionCol="prediction", metricName="f1")

#Compute Metrics
accuracy = accuracy_eval.evaluate(predictions)
precision = precision_eval.evaluate(predictions) #shows how many flagged transactions are actually fraud?
recall = recall_eval.evaluate(predictions) #shows how many fraud cases did we detect out of all
f1_score = f1_eval.evaluate(predictions)


print(f"Model Accuracy: {accuracy:.4f}")
print(f"Model Precision: {precision:.4f}")
print(f"Model Recall: {recall:.4f}")
print(f"Model F1-Score: {f1_score:.4f}")


25/03/06 09:14:56 WARN TaskSetManager: Stage 107 contains a task of very large size (5923 KiB). The maximum recommended task size is 1000 KiB.
25/03/06 09:14:56 WARN TaskSetManager: Stage 111 contains a task of very large size (5923 KiB). The maximum recommended task size is 1000 KiB.
25/03/06 09:14:57 WARN TaskSetManager: Stage 115 contains a task of very large size (5923 KiB). The maximum recommended task size is 1000 KiB.
25/03/06 09:14:57 WARN TaskSetManager: Stage 119 contains a task of very large size (5923 KiB). The maximum recommended task size is 1000 KiB.


Model Accuracy: 0.9860
Model Precision: 1.0000
Model Recall: 0.9860
Model F1-Score: 0.9930


### Exporting the Model Pipeline & Encoders

In [33]:
model_path = "hdfs://localhost:9000/datalake/raw/fraud-detection/"

In [34]:
pipeline_model.write().overwrite().save(f"{model_path}Classifier_Pipeline")
encoder.write().overwrite().save(f"{model_path}String_Indexer")